# Comparaison CSV bruts vs BDD star schema

Ce notebook trace le parcours des donnees depuis les 9 CSV bruts jusqu'aux 6 tables du star schema.
Trois angles d'analyse :

1. **Volumetrie** — combien de lignes/colonnes entrent, combien sortent
2. **Distributions avant/apres** — les transformations preservent-elles la coherence ?
3. **Perdu / Gagne** — ce qui disparait, ce qui est cree par l'ETL

## 0. Setup

In [ ]:
import sqlite3
import warnings

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from src.config import DATABASE_PATH
from src.etl.extract import load_all_raw

warnings.filterwarnings("ignore", category=FutureWarning)
sns.set_theme(style="whitegrid")
%matplotlib inline

In [ ]:
# Chargement des CSV bruts
raw = load_all_raw()

# Chargement des tables du star schema
conn = sqlite3.connect(str(DATABASE_PATH))
db = {}
for table in ["dim_dates", "dim_geolocation", "dim_customers", "dim_sellers", "dim_products", "fact_orders"]:
    db[table] = pd.read_sql(f"SELECT * FROM {table}", conn)
conn.close()

print(f"CSV : {len(raw)} datasets")
print(f"BDD : {len(db)} tables")

---
## 1. Volumetrie — le bilan des transformations

Combien de lignes et colonnes avant/apres l'ETL ?

In [ ]:
# Mapping CSV source → table cible
mapping = {
    "dim_geolocation": ["geolocation"],
    "dim_customers": ["customers"],
    "dim_sellers": ["sellers"],
    "dim_products": ["products", "category_translation"],
    "dim_dates": ["orders"],
    "fact_orders": ["order_items", "orders", "order_payments", "order_reviews"],
}

rows = []
for table, sources in mapping.items():
    src_rows = sum(raw[s].shape[0] for s in sources)
    src_cols = sum(raw[s].shape[1] for s in sources)
    dst_rows = db[table].shape[0]
    dst_cols = db[table].shape[1]
    ratio = f"{src_rows / dst_rows:.1f}:1" if dst_rows > 0 else "-"
    rows.append({
        "Table cible": table,
        "CSV sources": " + ".join(sources),
        "Lignes CSV": src_rows,
        "Lignes BDD": dst_rows,
        "Ratio": ratio,
        "Colonnes CSV": src_cols,
        "Colonnes BDD": dst_cols,
    })

vol = pd.DataFrame(rows).set_index("Table cible")
vol.style.format({"Lignes CSV": "{:,}", "Lignes BDD": "{:,}"})

In [ ]:
# Visualisation du ratio lignes CSV → BDD
fig, ax = plt.subplots(figsize=(10, 5))
x = range(len(vol))
width = 0.35

bars_csv = ax.bar([i - width/2 for i in x], vol["Lignes CSV"], width, label="CSV bruts", color="steelblue")
bars_bdd = ax.bar([i + width/2 for i in x], vol["Lignes BDD"], width, label="BDD star schema", color="coral")

ax.set_xticks(x)
ax.set_xticklabels(vol.index, rotation=30, ha="right")
ax.set_ylabel("Nombre de lignes")
ax.set_yscale("log")
ax.set_title("Volume de lignes : CSV bruts vs BDD (echelle log)")
ax.legend()

# Annotations ratio
for i, ratio in enumerate(vol["Ratio"]):
    ax.annotate(ratio, (i, max(vol.iloc[i]["Lignes CSV"], vol.iloc[i]["Lignes BDD"]) * 1.3),
                ha="center", fontsize=9, fontweight="bold")

plt.tight_layout()
plt.show()

### Lecture du bilan volumetrique

- **geolocation** subit la compression la plus forte (~53:1) : deduplication par zip_code_prefix
- **dim_dates** est **generee** a partir des timestamps de orders — le ratio n'a pas de sens direct
- **fact_orders** fusionne 4 CSV mais conserve le grain order_item (112k lignes) — les paiements et reviews sont agreges, pas les items
- **customers, sellers, products** restent proches du 1:1 — leur grain etait deja correct

---
## 2. Distributions avant/apres

Les transformations preservent-elles les distributions des donnees ?

### 2.1 Geolocation — coordonnees brutes vs medianes

In [ ]:
geo_raw = raw["geolocation"]
geo_db = db["dim_geolocation"]

# Bornes geographiques du Bresil — exclut les outliers GPS
LAT_BOUNDS = (-35, 6)
LNG_BOUNDS = (-75, -34)

raw_lat = geo_raw["geolocation_lat"]
raw_lng = geo_raw["geolocation_lng"]
db_lat = geo_db["lat"].dropna()
db_lng = geo_db["lng"].dropna()

# Filtrage
raw_lat_f = raw_lat[raw_lat.between(*LAT_BOUNDS)]
raw_lng_f = raw_lng[raw_lng.between(*LNG_BOUNDS)]
db_lat_f = db_lat[db_lat.between(*LAT_BOUNDS)]
db_lng_f = db_lng[db_lng.between(*LNG_BOUNDS)]

outliers_raw = len(raw_lat) - len(raw_lat_f)
outliers_db = len(db_lat) - len(db_lat_f)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Latitude — bins partages
axes[0].hist(raw_lat_f, bins=80, alpha=0.5, label=f"CSV brut ({len(geo_raw):,})", color="steelblue", density=True)
axes[0].hist(db_lat_f, bins=80, alpha=0.6, label=f"BDD mediane ({len(geo_db):,})", color="coral", density=True)
axes[0].set_title("Distribution des latitudes")
axes[0].set_xlabel("Latitude")
axes[0].set_xlim(LAT_BOUNDS)
axes[0].legend(fontsize=8)

# Longitude — bins partages
axes[1].hist(raw_lng_f, bins=80, alpha=0.5, label="CSV brut", color="steelblue", density=True)
axes[1].hist(db_lng_f, bins=80, alpha=0.6, label="BDD mediane", color="coral", density=True)
axes[1].set_title("Distribution des longitudes")
axes[1].set_xlabel("Longitude")
axes[1].set_xlim(LNG_BOUNDS)
axes[1].legend(fontsize=8)

plt.suptitle("Geolocation : 1M lignes brutes vs 19k medianes par zip (bornes Bresil)", fontweight="bold")
plt.tight_layout()
plt.show()

print(f"Outliers exclus — CSV: {outliers_raw}, BDD: {outliers_db}")
print("→ La forme globale est preservee. L'agregation par mediane lisse les outliers")
print("  sans deformer la distribution geographique sous-jacente.")

### 2.2 Paiements — valeurs brutes vs agregees

In [ ]:
pays_raw = raw["order_payments"]
fact = db["fact_orders"]

# payment_value brut (par ligne de paiement) vs order_payment_total (agrege par commande)
pays_agg_raw = pays_raw.groupby("order_id")["payment_value"].sum()
pays_agg_db = fact.drop_duplicates(subset="order_id")["order_payment_total"].dropna()

UPPER = 500
fig, axes = plt.subplots(1, 3, figsize=(16, 4))

# Distribution brute par ligne (filtree, pas clippee)
pays_raw.loc[pays_raw["payment_value"] <= UPPER, "payment_value"].plot.hist(
    bins=50, ax=axes[0], color="steelblue", edgecolor="white")
axes[0].set_title(f"CSV brut : payment_value par ligne\n({len(pays_raw):,} lignes)")
axes[0].set_xlabel(f"BRL (≤ {UPPER})")

# Distribution agregee depuis CSV
pays_agg_raw[pays_agg_raw <= UPPER].plot.hist(
    bins=50, ax=axes[1], color="mediumseagreen", edgecolor="white")
axes[1].set_title(f"CSV agrege : sum par commande\n({len(pays_agg_raw):,} commandes)")
axes[1].set_xlabel(f"BRL (≤ {UPPER})")

# Distribution dans la BDD
pays_agg_db[pays_agg_db <= UPPER].plot.hist(
    bins=50, ax=axes[2], color="coral", edgecolor="white")
axes[2].set_title(f"BDD fact_orders : order_payment_total\n({len(pays_agg_db):,} commandes uniques)")
axes[2].set_xlabel(f"BRL (≤ {UPPER})")

plt.suptitle("Paiements : de la ligne brute au total par commande", fontweight="bold")
plt.tight_layout()
plt.show()

pct_excl = (pays_agg_raw > UPPER).mean()
print(f"Commandes > {UPPER} BRL exclues de la visu : {pct_excl:.1%}")

In [ ]:
# Verification numerique : les totaux correspondent-ils ?
items_raw = raw["order_items"]

total_csv = pays_raw["payment_value"].sum()
total_db = fact.drop_duplicates(subset="order_id")["order_payment_total"].sum()

print(f"Total paiements CSV brut   : {total_csv:,.2f} BRL")
print(f"Total paiements BDD (dedup): {total_db:,.2f} BRL")
print(f"Ecart                      : {abs(total_csv - total_db):,.2f} BRL ({abs(total_csv - total_db)/total_csv:.4%})")

if abs(total_csv - total_db) < 1:
    print("\n→ Les totaux paiements concordent parfaitement.")
else:
    print(f"\n→ Ecart de {abs(total_csv - total_db):,.2f} BRL — a investiguer.")

# Verification prix et freight (order_items)
print()
for col in ["price", "freight_value"]:
    csv_sum = items_raw[col].sum()
    db_sum = fact[col].sum()
    print(f"{col:15s} — CSV: {csv_sum:>12,.2f}  BDD: {db_sum:>12,.2f}  ecart: {abs(csv_sum - db_sum):.2f}")

---
## 3. Perdu / Gagne — ce que l'ETL transforme

Bilan de ce qui disparait des CSV bruts et de ce que le star schema apporte.

In [ ]:
# Colonnes source (CSV) vs colonnes cible (BDD) par table
for table, sources in mapping.items():
    csv_cols = []
    for s in sources:
        csv_cols.extend([f"{s}.{c}" for c in raw[s].columns])
    bdd_cols = list(db[table].columns)

    print(f"{'─'*60}")
    print(f"  {table}  ←  {' + '.join(sources)}")
    print(f"  CSV : {len(csv_cols)} colonnes  →  BDD : {len(bdd_cols)} colonnes")
    print(f"  CSV : {', '.join(c.split('.')[-1] for c in csv_cols)}")
    print(f"  BDD : {', '.join(bdd_cols)}")
    print()

# Bilan global
all_csv = sum(raw[s].shape[1] for s in raw)
all_bdd = sum(db[t].shape[1] for t in db)
print(f"{'='*60}")
print(f"  Total : {all_csv} colonnes CSV  →  {all_bdd} colonnes BDD")

### Lecture du bilan

**Colonnes perdues** — supprimees car a faible valeur analytique :
- Textes creux : `review_comment_title` (88% vide), `review_comment_message` (59% vide)
- Identifiants techniques : `review_id`, `payment_sequential`
- Timestamps intermediaires : `order_approved_at`, `order_delivered_carrier_date`, etc. (utilises pour calculer `delivery_days` puis supprimes)
- Metadonnees derivees : `product_name_lenght`, `product_description_lenght`, `shipping_limit_date`
- Multi-lignes geolocation : 1M lignes reduites a 19k par agregation mediane/mode

**Colonnes gagnees** — creees par l'ETL :
- Cles surrogate et FK (`*_key`, `date_key`, `geo_key`) pour les jointures du star schema
- Metriques pre-calculees : `order_payment_total`, `delivery_days`, `estimated_days`, `delivery_delta_days`
- Traduction : `category_name_en` (fusion avec `category_translation`)
- Dimension temporelle : table `dim_dates` entierement generee (year, quarter, month, is_weekend)

---
## 4. Synthese

### Bilan de tracabilite

| Aspect | Constat |
|--------|--------|
| **Volumetrie** | 1.3M lignes CSV → 268k lignes BDD. La compression vient essentiellement de geolocation (53:1) et de l'agregation paiements/reviews |
| **Integrite numerique** | Les totaux price, freight et payment concordent entre CSV et BDD — aucune perte de donnees financieres |
| **Distributions** | Les distributions clefs (coordonnees, paiements) sont preservees apres transformation |
| **Colonnes perdues** | 16 colonnes a faible valeur analytique (textes vides, timestamps bruts, metadonnees) |
| **Colonnes gagnees** | 19 colonnes utiles (cles surrogate, FK, metriques derivees, traductions, dimension temporelle) |

### Conclusion

Le pipeline ETL transforme 9 CSV en 6 tables sans perte d'information analytique.
Les donnees supprimees sont soit redondantes (multi-lignes geolocation), soit a faible valeur (textes vides, metadonnees).
En retour, le star schema apporte des cles de jointure propres, des metriques pre-calculees et une dimension temporelle complete.

Voir aussi :
- [`docs/csv_to_star_schema.md`](../docs/csv_to_star_schema.md) — les choix de modelisation
- [`docs/exploration_analysis.md`](../docs/exploration_analysis.md) — l'analyse empirique des CSV bruts
- [`notebooks/exploration_csv.ipynb`](exploration_csv.ipynb) — l'exploration detaillee des CSV